In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPool2D,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import regularizers
from tqdm import tqdm

In [ ]:
traindir = "./dataset_final/Train"
validdir = "./dataset_final/Validation"
os.listdir(traindir)

In [ ]:
def show_image(dir):
    plt.figure(figsize=(12,7))
    img = load_img(os.path.join(dir))
    plt.imshow(img)

In [ ]:
height = 120
width = 120

train_datagen =  ImageDataGenerator(rescale = 1/255.0,rotation_range=45,height_shift_range=0.2,shear_range=0.2,
                              zoom_range=0.2,validation_split=0.2,horizontal_flip=True)

train_data_gray = train_datagen.flow_from_directory(directory = traindir,target_size=(height,width),
                                               color_mode='grayscale',
                                               class_mode = "categorical",batch_size=32,subset="training")

val_data_gray = train_datagen.flow_from_directory(directory = validdir,target_size=(height,width),
                                               class_mode = "categorical",batch_size=32,subset="validation")



test_datagen = ImageDataGenerator(rescale = 1/255.0)
test_data_gray = test_datagen.flow_from_directory(directory = validdir,target_size=(height,width),
                                               class_mode = "categorical",batch_size=32)



In [ ]:
def normalize_im(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalized data
    """
    return np.array((x - np.min(x)) / (np.max(x) - np.min(x)))

In [ ]:
from PIL import Image
import os, os.path

imgs = []
path = "./dataset_final/Train/Male"
valid_images = [".jpg",".gif",".png",".tga"]
for f in tqdm(os.listdir(path)):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = Image.open(os.path.join(path,f)).convert('L')
    im = np.array(im)

    my_image = im
    my_image = color_grayscale_arr_v2(my_image,"green")
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)
    my_image = my_image.reshape((my_image.shape[1], my_image.shape[2],3))
    my_image = normalize_im(my_image)

    plt.imsave("./dataset_final/Train_Copy/Male/"+f, my_image)

#     imgs.append(Image.open(os.path.join(path,f)))

In [ ]:
from PIL import Image
import os, os.path

imgs = []
path = "./dataset_final/Train/Female"
valid_images = [".jpg",".gif",".png",".tga"]
for f in tqdm(os.listdir(path)):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = Image.open(os.path.join(path,f)).convert('L')
    im = np.array(im)

    my_image = im
    my_image = color_grayscale_arr_v2(my_image,"green")
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)
    my_image = my_image.reshape((my_image.shape[1], my_image.shape[2],3))
    my_image = normalize_im(my_image)

    plt.imsave("./dataset_final/Train_Copy/Female/"+f, my_image)

#     imgs.append(Image.open(os.path.join(path,f)))

In [ ]:
def color_grayscale_arr_v2(arr, color='white'):
    """Converts grayscale image to either red or green"""
    assert arr.ndim == 2
    dtype = arr.dtype
    h, w = arr.shape
    arr = np.reshape(arr, [h, w, 1])
    if (color=='red'):
        arr = np.concatenate([arr,
                          np.zeros((h, w, 2), dtype=dtype)], axis=2)
    elif (color=='green'):
        arr = np.concatenate([np.zeros((h, w, 1), dtype=dtype),
                          arr,
                          np.zeros((h, w, 1), dtype=dtype)], axis=2)
    elif (color=='blue'):
        arr = np.concatenate([np.zeros((h, w, 1), dtype=dtype),
                          np.zeros((h, w, 1), dtype=dtype), 
                         arr], axis=2)
    elif (color=='white'):
        arr = np.concatenate([arr,
                          arr, 
                         arr], axis=2)
        
    return arr

In [ ]:
def color_grayscale_arr(arr, color='white'):
    """Converts grayscale image to either red or green"""
    assert arr.ndim == 3
    dtype = arr.dtype
    h, w,_ = arr.shape
    arr = np.reshape(arr, [h, w, 1])
    if (color=='red'):
        arr = np.concatenate([arr,
                          np.zeros((h, w, 2), dtype=dtype)], axis=2)
    elif (color=='green'):
        arr = np.concatenate([np.zeros((h, w, 1), dtype=dtype),
                          arr,
                          np.zeros((h, w, 1), dtype=dtype)], axis=2)
    elif (color=='blue'):
        arr = np.concatenate([np.zeros((h, w, 1), dtype=dtype),
                          np.zeros((h, w, 1), dtype=dtype), 
                         arr], axis=2)
    elif (color=='white'):
        arr = np.concatenate([arr,
                          arr, 
                         arr], axis=2)
        
    return arr

In [ ]:
def generate_colored_data(init_dataset_gray,color,nb_samples):
    samples = []
    labels = []
    count = 0
    for gray_sample in train_data_gray:
        my_image = gray_sample[0][0]
        label = gray_sample[1][0]
        my_image = color_grayscale_arr(my_image,color)
        my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
        my_image = preprocess_input(my_image)
        my_image = my_image.reshape((my_image.shape[1], my_image.shape[2],3))
        samples.append(my_image)
        labels.append(label)
        count = count + 1
        if(count == nb_samples):
            break
    return np.array(samples),np.array(labels)

In [ ]:
nb_pics = 18000

X_gray_samples, Y_gray_samples = generate_colored_data(train_data_gray,"white",nb_pics)
print("Done Gray")
X_green_samples, Y_green_samples = generate_colored_data(train_data_gray,"green",nb_pics)
print("Done Green")
X_blue_samples, Y_blue_samples = generate_colored_data(train_data_gray,"blue",nb_pics)
print("Done Blue")
X_red_samples, Y_red_samples = generate_colored_data(train_data_gray,"red",nb_pics)
print("Done Red")

In [ ]:
X_gray_samples.shape
Y_gray_samples.shape

In [ ]:
model = Sequential()
model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(height,width,3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(2,activation = "sigmoid"))

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss = "categorical_crossentropy",metrics =["accuracy"])
model.summary()

In [ ]:
# train_data.samples
batch_size = 32
len(X_green_samples)// batch_size

In [ ]:
len(X_green_samples)

In [ ]:
history = model.fit(X_green_samples, Y_green_samples, epochs=10,batch_size=256, validation_split=0.1)

In [ ]:
for i in range(len(model.layers)):
    layer = model.layers[i]
    
    print(i, layer.name, layer.output.shape)

In [ ]:
from tensorflow.keras import models

layer_outputs = [layer.output for layer in model.layers[:12]] 
# Extracts the outputs of the top 12 layers
activation_model = models.Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input

In [ ]:
# This func will compute layerwise average activations and the maximum of these means
def compute_avg_activation(activations,normalize=False):
    nb_layers = len(activations)
    layers_means = []
    lambdas = []
    for l in range(nb_layers):
        layer_output = activations[l][0]
        layer_mean_of_activs = [] # Stores average activations per layer
        for i in range (layer_output.shape[2]): # loop over each filter (3rd dimention) 
            filter_mean_activ = layer_output[:,:,i].mean() # Get the mean (A hat, from the paper)
            layer_mean_of_activs.append(filter_mean_activ)
        layer_max_activ = max(layer_mean_of_activs) # Get max (This is lambda of the layer)
        lambdas.append(layer_max_activ)
        if(normalize):
            layer_mean_of_activs = [x/layer_max_activ for x in layer_mean_of_activs] # Normalize
        layers_means.append(layer_mean_of_activs) # This is lambda prime of layer
    return layers_means, lambdas

In [ ]:
# Dataset should be for one target ethnicity 
def compute_lambdas(test_samples):
    lambdas = []
    for i in tqdm(range(len(test_samples))):
        image = test_samples[i]
        image = np.reshape(image,[1,120,120,3])
        all_outputs = activation_model.predict(image)
        means,layers_lambdas = compute_avg_activation(all_outputs,True)
        lambdas.append(layers_lambdas)
    lambdas = np.array(lambdas)
    lambdas_max = np.max(lambdas,0) # store the max per layer
    lambdas_min = np.min(lambdas,0) # store the min per layer
    return lambdas_max,lambdas_min

In [ ]:
lambdas_max_gray_samples, lambdas_min_gray_samples = compute_lambdas(gray_samples)

In [ ]:
lambdas_max_colored_samples, lambdas_min_colored_samples = compute_lambdas(colored_samples)

In [ ]:
lambdas_max_red_samples, lambdas_min_red_samples = compute_lambdas(red_samples)

In [ ]:
lambdas_max_blue_samples, lambdas_min_blue_samples = compute_lambdas(blue_samples)

In [ ]:
lambdas_max_green_samples, lambdas_min_green_samples = compute_lambdas(green_samples)

In [ ]:
layers = [x for x in range(12)]

In [ ]:
# from matplotlib.pyplot import figure

# figure(figsize=(8, 6), dpi=80)

# plt.plot(layers, lambdas_max_colored_samples,'orange')
# plt.plot(layers, lambdas_max_gray_samples,'black')

# plt.plot(layers, lambdas_max_red_samples,'red')
# plt.plot(layers, lambdas_max_green_samples,'green')
# plt.plot(layers, lambdas_max_blue_samples,'blue')

# plt.legend(['Colored',"Gray","Red","Green","Blue"])
# plt.title('Activations per Category')
# plt.xlabel('Layer')
# plt.ylabel('Activation Norm')
# plt.show()

In [ ]:
from numpy import inf

vals_red = lambdas_max_red_samples/lambdas_min_red_samples
vals_red[vals_red==inf] = 1.0

vals_colored = lambdas_max_colored_samples/lambdas_min_colored_samples
vals_colored[vals_colored==inf] = 1.0

vals_gray = lambdas_max_gray_samples/lambdas_min_gray_samples
vals_gray[vals_gray==inf] = 1.0

vals_green = lambdas_max_green_samples/lambdas_min_green_samples
vals_green[vals_green==inf] = 1.0

vals_blue = lambdas_max_blue_samples/lambdas_min_blue_samples
vals_blue[vals_blue==inf] = 1.0

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

def normal(arr):    
    x = arr
    norm1 = x / np.linalg.norm(x)
    norm2 = normalize(x[:,np.newaxis], axis=0).ravel()
    return norm2


In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)

plt.plot(layers, normal(vals_colored),'orange')
# plt.plot(layers, normal(vals_gray),'black')

plt.plot(layers, normal(vals_red),'red')
plt.plot(layers, normal(vals_green),'green')
plt.plot(layers, normal(vals_blue),'blue')

plt.legend(['Colored',"Red","Green","Blue"])
plt.title('Activations per Category')
plt.xlabel('Layer')
plt.ylabel('Activation Norm')
plt.show()

In [ ]:
# normal(vals_gray)